# run experiments
for each dataset, for each group size and for each group
get items from the mf for each member of the group
these items will be the input of the algorithm

In [ ]:
import numpy as np
import pandas as pd
from typing import List


u_features = np.load('../datasets/movie_lens/mf/U_features.npy')
i_features = np.load('../datasets/movie_lens/mf/I_features.npy')
print(u_features.shape)
print(i_features.shape)

def get_items_for_user(user_id):
    items_ratings = u_features[:, user_id] @ i_features
    items_ids_w_ratings = [(item_id, rating) for item_id, rating in enumerate(items_ratings)]
    items_ids_w_ratings.sort(key=lambda x: x[1], reverse=True)
    return items_ids_w_ratings

def get_items_for_users(users_id: List):
    items_ratings = i_features.T @ u_features[:, users_id]
    return items_ratings
    
ratings = get_items_for_users([10,20,30])
ratings.shape


In [ ]:
from collections import defaultdict
import itertools
import math

from tqdm import tqdm


def select_top_n(score_list, top_n):
    top_n_ind = np.argpartition(score_list, -top_n)[-top_n:]
    sorted_top_n_indicies = top_n_ind[np.argsort(-score_list[top_n_ind])]
    return sorted_top_n_indicies

def get_idx_of_top_n(score_list, top_n):
    """Returns the indices of the top_n items"""
    top_n_idx = np.argpartition(score_list, -top_n)[-top_n:]
    return top_n_idx

def avg_algorithm(group_items, top_n: int):
    """
    Returns items ordered by average rating.
    """
    means = group_items.mean(axis=1)
    top_n_idx = select_top_n(means, top_n)
    return top_n_idx

def lm_algorithm(group_items, top_n: int):
    """
    Returns items ordered by least min value across user rating.
    """
    mins = group_items.min(axis=1)
    top_n_idx = select_top_n(mins, top_n)
    return top_n_idx

def fai_algorithm(group_items, top_n: int):
    """
    Returns items ordered by max of users each one by one per turn.
    So first item is selected as max of first user, second item by second and so on...
    """
    group_size = group_items.shape[0]
    # apply select_top_n to each user
    top_n_required_per_user = math.ceil(top_n / group_size)
    top_n_idx_per_user = np.apply_along_axis(lambda row: select_top_n(row, top_n_required_per_user), 1, group_items)
    # flatten the list to get the turn by turn top_n_idx
    top_n = top_n_idx_per_user.flatten(order='F')[:top_n]
    return top_n

def get_rank(score_list_np):
    """Best item has a rank of 1"""
    ranks = np.zeros(score_list_np.shape, dtype=np.int32)
    ranks[score_list_np.argsort()] = np.arange(start=score_list_np.shape[0], stop=0, step=-1)
    return ranks


group_size = 5

# load groups
groups = pd.read_csv('../notebooks/dfs/groups/kgrec/top_k_10.csv')
#concatenate first 5 columns to array of ints
groups = groups.iloc[:,:group_size].values

rec_it_avg = []
rec_it_lm = []
rec_it_fai = []

for group_members in tqdm(groups):
    items = get_items_for_users(group_members)

    # avg_algorithm
    top_n_items_avg = avg_algorithm(items, 10)
    rec_it_avg.append(top_n_items_avg)

    # lm_algorithm
    top_n_items_lm = lm_algorithm(items, 10)
    rec_it_lm.append(top_n_items_lm)

    # fai_algorithm
    top_n_items_fai = fai_algorithm(items, 10)
    rec_it_fai.append(top_n_items_fai)